In [26]:
import requests
import json
import pandas as pd
import numpy as np

In [42]:
url = 'http://apis.data.go.kr/1360000/FmlandWthrInfoService/getDayStatistics'
servicekey_en = 'f8%2FAQMI%2FVCCLb%2FOjO4hPfmZwE9qdtIrRD07dhQgRjR9OjCkyAsjEzJIBm7raIgOlA4GCw%2FGJp7bytgeT2lYcrw%3D%3D'
servicekey_de = 'f8/AQMI/VCCLb/OjO4hPfmZwE9qdtIrRD07dhQgRjR9OjCkyAsjEzJIBm7raIgOlA4GCw/GJp7bytgeT2lYcrw=='

In [43]:
numOfrows = '10000' #페이지당 결과물 출력 수
start_date = '20100101' #조회 시작일 (2010년 1월 1일)
end_date = '20210930' #조회 마지막일(2021년 10월 1일)
area_id = '999999999' #조회 지역 코드 , 999999999 입력시 전체지역
product_id = 'PA999999' #주산지 작물별 특성 아이디(PA999999 입력시 전체 작물)

In [44]:
# *대상품목(16): 배추, 무, 양파, 건고추, 마늘, 대파, 얼갈이배추, 양배추, 깻잎, 시금치, 미나리, 당근, 파프리카, 새송이, 팽이버섯, 토마토
# *대상품종(5): 청상추, 백다다기, 애호박, 캠벨얼리, 샤인마스캇

In [45]:
# product_key = {배추 : 'PA170401', 무 : 'PA130201', 양파 : 'PA250201', 건고추 : 'PA030101', 마늘 : 'PA120101', 대파 : 'PA090101',
#               얼갈이배추 : 'PA260101', 양배추 : 'PA240101', 깻잎 : 'PA050101', 시금치 : 'PA220101', 미나리 : 'PA140101',
#               당근: 'PA080101', 캠벨얼리 : 'PA340101'} #파프리카, 새송이 팽이버섯 토마토, 청상추, 백다다기, 애호박, 샤인마스캇은  전처리 후 , 지역이름을 따야할듯


In [46]:
#영천 포도-캠벨얼리
params ={'serviceKey' : servicekey_de, 'pageNo' : '1', 'numOfRows' : numOfrows , 'dataType' : 'JSON', 'ST_YMD' : start_date, 'ED_YMD' : end_date, 'AREA_ID' : area_id, 'PA_CROP_SPE_ID' : product_id }

In [47]:
response = requests.get(url, params=params)

In [48]:
data_api = json.loads(response.text) #딕셔너리 타입으로 나옴

In [49]:
data_api['response']['body']['totalCount']  #전체 데이터 갯수

817556

In [38]:
dict_data = data_api['response']['body']['items']['item']   # 데이터를 데이터 프레임에 넣기
product_data = pd.DataFrame(dict_data)

In [39]:
product_data.shape

(1000, 16)

In [40]:
numOfpage = data_api['response']['body']['totalCount']// 10000 #데이터 총량을 만단위로 나눈 몫
numOfpage

817

In [41]:
for i in range(2,numOfpage+2):
    params ={'serviceKey' : servicekey_de, 'pageNo' : str(i), 'numOfRows' : numOfrows , 'dataType' : 'JSON', 'ST_YMD' : start_date, 'ED_YMD' : end_date, 'AREA_ID' : area_id, 'PA_CROP_SPE_ID' : product_id }
    response = requests.get(url, params=params)
    print(i)
    data_api_add = json.loads(response.text) #딕셔너리 타입으로 나옴
    dict_data_add = data_api_add['response']['body']['items']['item']
    product_data_add = pd.DataFrame(dict_data_add)
    product_data = pd.concat([product_data,product_data_add], axis = 1)

2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
product_data.shape

In [ ]:
#시계열로 나누어진 엑셀 파일을 열때마다 동시계열의 api 호출을 해서 데이터 크기를 줄여야함. 너무 크면 호출 실패